<a href="https://colab.research.google.com/github/gibsonx/jlpt_simulator/blob/dev/graphs/n3/outliner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
if 'google.colab' in str(get_ipython()):
    !git clone https://github.com/gibsonx/jlpt_simulator.git
    %cd jlpt_simulator
    !git checkout dev
    
    !apt-get install python3-dev graphviz libgraphviz-dev pkg-config
    !pip install -r requirements.txt
else:
  print('Not running on CoLab')


Not running on CoLab


In [2]:
import json
import logging
import random
import time
from spire.doc import *
from spire.doc.common import *
import pandas as pd
import yaml
from tqdm import tqdm

# from langchain_community.embeddings import XinferenceEmbeddings
from libs.LLMs import *
import datetime
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

from graphs.n3.ExamTaskHandler import ExamTaskHandler
load_dotenv()

True

In [3]:
def collect_vocabulary(file_path):
    # Read the CSV file
    data = pd.read_csv(file_path)
    # Shuffle the rows and reset the index
    words = data.iloc[:, :2].sample(frac=1).reset_index(drop=True)
    # Extract the second column (values) and convert to a single-line string
    vocab_string = ','.join(words.iloc[:, 1].astype(str).tolist())
    return vocab_string

In [4]:
# Import N3 Vocabulary
file_path = 'Vocab/n3.csv'
# Display the content of the CSV file
vocab_dict = collect_vocabulary(file_path)
with open("Vocab/topics.txt", "r", encoding="utf-8") as file:
    topics_list = [line.strip() for line in file]

In [5]:
vocab_dict

'かいけつ,ほぞん,そう,しょくりょう,ざいさん,はん,く,もっとも,しょうたい,とどく,はんだん,さく,かいしゃく,そう,もやす,せんこう,かしこい,はか,さほう,あつかう,じゅうし,きたい,ず,ちる,おろす,あと,こううん,おおいに,えがお,じつりょく,で,さいしゅう,ゆうしゅう,かげん,ぬの,つぐ,きょうじゅ,たんじょう,うわさ,ごうけい,こくみん,ふく,あたり,いいん,な,きゅうけい,ゆるす,みかた,ほり,しんたい,こおる,ねんれい,たま,おうじる,よのなか,どうよう,む,かる,つかまる,つめる,きょだい,しぼう,そうとう,とうちゃく,りゅうがく,むかえ,うったえる,せいちょう,せつ,かげ,きり,しらせ,かえす,せい,まぜる,ひたい,のる,うまれ,かくご,さら,はかせ,けむり,おく,ふくめる,きげん,ほね,じょう,しんこう,かんりょう,のこす,ひひょう,とうあん,ちょうし,けいい,えさ,しどう,むだ,たおす,たい,かん,つり,かぐ,ぜいたく,らい～,かおり,ちく,しまい,かんり,ちい,しきりに,あんがい,きゅう,こうふん,ひげき,ふ,くるう,おだやか,なぞ,つぐ,しょうりゃく,ようす,ジェットき,しょう,ゆずる,ふくむ,てんねん,のぞく,かがく,しょうご,いや,けんとう,さんか,いんたい,おび,がら,しゅうしょく,ふあん,いきもの,いた,けんこう,りえき,ひみつ,ごみ,かたづく,ようき,たいおん,ひやす,よほう,すがた,やね,しんこう,かんそう,おとなしい,しょうがい,はし,けしょう,こうりょ,じょうけん,とうじ,ひとしい,ねんかん,あかり,くだる,うわ,よる,かく,しゅふ,ひろがる,いき,ひとり,こうはい,けいえい,たつ,れいぎ,とばす,もくひょう,さんせい,お,みやげ,かんじる,ろうどう,かいぜん,しゅうせい,やくす,とける,りく,えんそう,しゅだん,まえもって,そで,どくしん,けつえき,たんなる,とくい,かがやく,ふえ,むく,たしょう,かてい,だいぶぶん,けいゆ,のぞむ,ひづけ,うれる,ようじん,とう,およぎ,ひっこし,おぼれる,きょうりょく,いっしゅん,そだつ,いじ,おじぎ,ぶんや,おろす,じゅん,じき,かいがい,ねがう,しげき,いっち,しぜん,ちきゅう,ほこり,ぜん,どく,めし,ちかごろ,うつる,としょ,ねんだい,しゅう,けんかい,ぶ,かんげい,ま

#### load Models

In [6]:
from typing import List, Optional

from langchain_core.prompts import ChatPromptTemplate

from pydantic import BaseModel, Field

# instruction = """
# Section 1: Vocabulary and Grammar
# - Kanji reading (kanji_reading): 8 questions
# - Write Chinese characters (write_kanji): 6 questions
# - Word Meaning Selection (word_meaning): 11 questions
# - Synonyms substitution (synonym_substitution): 5 questions
# - word usage (word_usage): 5 questions
# - Grammar fill in the blank (sentence_grammar): 13 questions
# - Sentence sorting (sentence_sort): 5 questions
# - Grammar structure selection (sentence_structure): 4-5 questions
#
# Section 2: Reading Comprehension
# - Short passages (short_passage_read): 4 articles
# - Mid-size passages (midsize_passage_read): 2 articles
# - Long passages (long_passage_read): 1 articles
# - Information retrieval (info_retrieval): 1 articles
#
# Section 3: Listening Comprehension
# - Topic understanding (topic_understanding): 6 questions
# - Key understanding (keypoint_understanding): 6 questions
# - Summary understanding (summary_understanding): 3 questions
# - Active expression (active_expression): 4 questions
# - Immediate acknowledgment (immediate_ack): 9 questions
# """

# '''
# - 問題1 のことばの読み方として最もよいものを、1・2・3・4から一つえらびなさい (kanji_reading): 1 question
# - 問題2 このことばを漢字で書くとき、最もよいものを、１・２・３・４から一つえらびなさい (write_kanji): 1 question
# - 問題3（　）に入れるのに最もよいものを、１・２・３・４から一つえらびなさい。 (word_meaning): 1 question
# - 問題4 に意味が最も近いものを、１・２・３・４から一つえらびなさい。(synonym_substitution) 1 question
# - 問題5 つぎのことばの使い方として最もよいものを、２・３・４から一つえらびなさい。 (word_usage): 1 question
# - 問題6 つぎの文の（　　　）に入れるのに最もよいものを、１・２・３・４から一つえらびなさい。(sentence_grammar) 1 questions
# - 問題7 つぎの文の ★ に入る最もよいものを、１・２・３・４から一つえらびなさい。(sentence_sort) 1 question
# '''

instruction = """
Section 1: Vocabulary and Grammar
- 問題8 つぎの文章を読んで、文章全体の内容を考えて、文中の 19 から 22 の中に入る最もよいものを、１・２・３・４から一つえらびなさい (sentence_structure) 1 question

Section 2: Reading Comprehension
- 問題9 つぎの(1)から(4)の文章を読んで、質問に答えなさい。答えは、1・2・3・4から最もよいものを一つえらびなさい (short_passage_read): 4 articles
- Information retrieval (info_retrieval): 1 article

Section 3: Listening Comprehension
- Topic understanding (topic_understanding): 1 question

"""


direct_gen_outline_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            f"You are a japanese teacher. Your job is to write an outline for a JLPT Japanese-Language Proficiency Test N3 level exam paper. The topic word in outline must be presented in Japanese kana only. The complexity should be restricted to N3 level and respect japanese culture. The JLPT Japanese-Language Proficiency Test exam paper includes a mix of moderate and very difficult topics to accurately assess the test-taker's proficiency across different aspects of the language."
            f"First, evenly pick Japanese words used in different part of sentence from 'Vocabulary' for questions in Vocabulary and Grammar, ensuring 40 percent of very difficult words for topics. At Section 2 and 3, randomly choose topics in 'TopicList' for reading Comprehension and Listening Comprehension Sections, don't repeat to choose a same word or topic"
            f"Second, you should abide by the provided exam instruction and decide the number of questions and content in the each Section."
            f"Finally, write the outline of the examination paper in japanese and provide question topics according to the instructions."
            f"Instruction: {instruction}",
        ),
        ("user", "TopicList: {topic_list}, Vocabulary: {vocab_dict}"),
    ]
)


## Data Strcuture

In [7]:
class QuestionTopic(BaseModel):
    topic: str = Field(..., title="a vocabulary or topic hint for a question")


class Subsection(BaseModel):
    subsection_title: str = Field(..., title="subsection English word in () only from the instruction")
    description: str = Field(..., title="giving the number of questions and requirements")
    question_topics: Optional[List[QuestionTopic]] = Field(
        default_factory=list
    )

    @property
    def as_str(self) -> str:
        question_topics_str = "\n".join(
            f"- **{qt.topic}**" for qt in self.question_topics
        )
        return f"### {self.subsection_title}\n\n{self.description}\n\n{question_topics_str}".strip()

class Section(BaseModel):
    section_title: str = Field(..., title="Title of the section")
    subsections: Optional[List[Subsection]] = Field(
        default_factory=list,
        title="Titles and reason for each subsection of the JLPT exam page.",
    )

    @property
    def as_str(self) -> str:
        subsections = "\n\n".join(
            subsection.as_str for subsection in self.subsections or []
        )
        return f"## {self.section_title}\n\n{subsections}".strip()


class Outline(BaseModel):
    page_title: str = Field(..., title="Title of the JLPT exam page")
    sections: List[Section] = Field(
        default_factory=list,
        title="Titles and descriptions for each section of the JLPT exam paper.",
    )

    @property
    def as_str(self) -> str:
        sections = "\n\n".join(section.as_str for section in self.sections)
        return f"# {self.page_title}\n\n{sections}".strip()


In [8]:
# Read the topics from a file, sort them, and print the sorted list
def process_topics(file_path):
    try:
        # Read the file
        with open(file_path, 'r', encoding='utf-8') as file:
            topics = file.readlines()

        # Remove any extra whitespace or newline characters
        topics = [topic.strip() for topic in topics if topic.strip()]

        # Shuffle the topics randomly
        random.shuffle(topics)

    except FileNotFoundError:
        print("The file was not found. Please check the file path.")
    except Exception as e:
        print("An error occurred:", str(e))

    except FileNotFoundError:
        print("The file was not found. Please check the file path.")
    except Exception as e:
        print("An error occurred:", str(e))

In [9]:
# Preload all topics from the file
with open("Vocab/topics.txt", "r", encoding="utf-8") as file:
    topics_list = [line.strip() for line in file]

generate_outline_direct = direct_gen_outline_prompt | azure_llm.with_structured_output(Outline)
initial_outline = generate_outline_direct.invoke({"topic_list": topics_list, "vocab_dict": vocab_dict})

2025-06-09 22:35:08,513 - INFO - HTTP Request: POST https://ai-rolandaws880125ai409947751408.openai.azure.com/openai/deployments/gpt-4.1/chat/completions?api-version=2025-01-01-preview "HTTP/1.1 200 OK"


In [10]:
initial_outline

Outline(page_title='日本語能力試験N3 模擬試験 問題用紙', sections=[Section(section_title='第１部\u3000語彙・文法', subsections=[Subsection(subsection_title='(sentence_structure)', description='問題8：1問。つぎの文章を読んで、文章全体の内容を考えて、文中の 19 から 22 の中に入る最もよいものを、１・２・３・４から一つえらびなさい。', question_topics=[QuestionTopic(topic='しゅうしょく'), QuestionTopic(topic='かいしゃく'), QuestionTopic(topic='ふくむ'), QuestionTopic(topic='じょうけん')])]), Section(section_title='第２部\u3000読解', subsections=[Subsection(subsection_title='(short_passage_read)', description='問題9：4つの短い文章を読んで、それぞれの質問に答えなさい。答えは、1・2・3・4から最もよいものを一つえらびなさい。', question_topics=[QuestionTopic(topic='食事の好みについて話す'), QuestionTopic(topic='道を尋ねる'), QuestionTopic(topic='家族について話す'), QuestionTopic(topic='健康とフィットネスについて話す')]), Subsection(subsection_title='(info_retrieval)', description='1つの記事を読んで、情報を探し出し、最もよい答えを選びなさい。', question_topics=[QuestionTopic(topic='引っ越しの準備について話す')])]), Section(section_title='第３部\u3000聴解', subsections=[Subsection(subsection_title='(topic_understanding)', description='1問。音声を聞き、

In [11]:
from IPython.display import display, Markdown, HTML
display(Markdown(initial_outline.as_str))

# 日本語能力試験N3 模擬試験 問題用紙

## 第１部　語彙・文法

### (sentence_structure)

問題8：1問。つぎの文章を読んで、文章全体の内容を考えて、文中の 19 から 22 の中に入る最もよいものを、１・２・３・４から一つえらびなさい。

- **しゅうしょく**
- **かいしゃく**
- **ふくむ**
- **じょうけん**

## 第２部　読解

### (short_passage_read)

問題9：4つの短い文章を読んで、それぞれの質問に答えなさい。答えは、1・2・3・4から最もよいものを一つえらびなさい。

- **食事の好みについて話す**
- **道を尋ねる**
- **家族について話す**
- **健康とフィットネスについて話す**

### (info_retrieval)

1つの記事を読んで、情報を探し出し、最もよい答えを選びなさい。

- **引っ越しの準備について話す**

## 第３部　聴解

### (topic_understanding)

1問。音声を聞き、内容理解を問う問題。

- **公共施設の利用方法について話す**

In [12]:
import re
outliner_json = initial_outline.model_dump_json()
data = json.loads(outliner_json)  # Replace with your actual JSON data

start_time = time.time()

output_data = {'sections': []}

for section in data['sections']:
    output_section = {'section_title': section['section_title'], 'subsections': []}

    for subsection in tqdm(section['subsections'], desc=f"Processing {section['section_title']}"):
        function_name = re.findall(r'\((.*?)\)', subsection['subsection_title'])[0]

        questions = subsection['question_topics']

        for question in tqdm(questions, desc=f"Processing {subsection['subsection_title']}"):
            handler = ExamTaskHandler()
            func = getattr(handler, function_name, None)

            if func:
                max_attempts = 2
                original_topic = question['topic']  # Optional: Track original topic
                for attempt in range(max_attempts):
                    try:
                        result = func(question['topic'])
                        question['result'] = result
                        break  # Exit on success
                    except Exception as e:
                        if attempt < max_attempts - 1:
                            # Replace topic and retry
                            question['topic'] = random.choice(topics_list)
                        else:
                            question['result'] = f"Error after {max_attempts} attempts: {str(e)}"
            else:
                question['result'] = f"Method {function_name} not found"

        output_subsection = {
            'subsection_title': subsection['subsection_title'],
            'description': subsection['description'],
            'question_topics': questions
        }
        output_section['subsections'].append(output_subsection)

    output_data['sections'].append(output_section)

# End the timer
end_time = time.time()

# Calculate the total execution time
execution_time = end_time - start_time

print(f"Total execution time: {execution_time:.2f} seconds")

Processing (sentence_structure):   0%|          | 0/4 [00:00<?, ?it/s]

---WEB SEARCH---
---Generator----


2025-06-09 22:35:20,363 - INFO - HTTP Request: POST https://ai-rolandaws880125ai409947751408.openai.azure.com/openai/deployments/gpt-4.1/chat/completions?api-version=2025-01-01-preview "HTTP/1.1 200 OK"
2025-06-09 22:35:20,364 - INFO - Generated message: <div class='article'>
<h3>初めての就職活動</h3>
<p>
私は大学三年生の時、初めて<strong>就職活動</strong>を始めました。就職活動はとても大変だと聞いていましたが、<strong>【19】</strong>、自分の将来について考える良いきっかけにもなりました。
</p>
<p>
最初はどの会社に応募したらいいか分からず、インターネットでいろいろな企業について調べました。説明会に参加したり、先輩にアドバイスをもらったりして、少しずつ自分に合う会社が分かってきました。<strong>【20】</strong>、面接はとても緊張しましたが、何度も練習することで自信がつきました。
</p>
<p>
私は「人の役に立つ仕事がしたい」と思っていたので、社会福祉の会社に応募しました。面接では、自分の考えや経験を<strong>【21】</strong>話すように心がけました。
</p>
<p>
最終的に、希望していた会社から内定をもらうことができました。今は社会人として働きながら、毎日新しいことを学んでいます。就職活動は大変でしたが、<strong>【22】</strong>、とても良い経験になったと思います。
</p>
</div>

<a>19.</a>
<ul>
    <li>もちろん</li>
    <li>けれども</li>
    <li>たとえば</li>
    <li>それとも</li>
</ul>
正解: 2

<a>20.</a>
<ul>
    <li>ですから</li>
    <li>ところが</li>
    <li>たしかに</li>
    <li>それにしては</li>
</ul>
正解: 

---REVISOR---


2025-06-09 22:35:26,324 - INFO - HTTP Request: POST https://ai-rolandaws880125ai409947751408.openai.azure.com/openai/deployments/gpt-4.1/chat/completions?api-version=2025-01-01-preview "HTTP/1.1 200 OK"
2025-06-09 22:35:26,325 - INFO - Refelect message: **Content Accuracy:**
- The reading passage is clear, relevant, and appropriately challenging for JLPT N3. The vocabulary and grammar used are suitable for this level (e.g., 就職活動, 内定, けれども, たしかに, できるだけ, なるべく, 苦労したけれど).
- The context of job hunting as a university student is both culturally and academically appropriate for Japanese learners.

**Question and Answer Quality:**
- All questions are clearly worded and unambiguous. The blanks test logical connectors and adverbs, which are essential at N3.
- The distractors in each question are plausible, but only one is contextually and grammatically correct, which is ideal.
- The difficulty level matches the intended N3 level, as students must understand both vocabulary nuance and logical flo

...

---REVISOR---


2025-06-09 22:38:29,806 - INFO - HTTP Request: POST https://ai-rolandaws880125ai409947751408.openai.azure.com/openai/deployments/gpt-4.1/chat/completions?api-version=2025-01-01-preview "HTTP/1.1 200 OK"
2025-06-09 22:38:29,808 - INFO - Refelect message: **Content Accuracy:**  
The dialogue and question are appropriate for the JLPT N3 level. The vocabulary (e.g., 申込書, 利用方法, 受付, 必要事項), grammar (e.g., どうすればいいですか, 〜してください), and context (public facility usage) are all suitable for N3. The reading passage is clear, relevant, and appropriately challenging, requiring inference based on the exchange.

**Question and Answer Quality:**  
The question, 「女の人はこの後、何をしますか。」, is clear and direct. The answer choices are plausible but only one is correct, which tests the student's comprehension and inference skills. The distractors are logical and not confusing. The correct answer is (3) 利用方法を説明する, as in the dialogue, the woman is currently explaining the usage procedure.

**Detailed Refinement:**  
- Th

--- AI Reviser feels Good Enough ---
--- Formatter ---


2025-06-09 22:38:33,332 - INFO - HTTP Request: POST https://ai-rolandaws880125ai409947751408.openai.azure.com/openai/deployments/gpt-4.1/chat/completions?api-version=2025-01-01-preview "HTTP/1.1 200 OK"
2025-06-09 22:38:33,334 - INFO - Formatted message: {'question': {'html_question': '公民館で男の人と女の人が話しています。女の人はこの後、何をしますか。', 'correct_answer': 3, 'choices': ['会議室のそうじをする', '申込書を受付に出す', '利用方法を説明する', '机や椅子をならべる']}, 'conversation': [{'gender': 'male', 'context': 'すみません、公民館の会議室を利用したいのですが、どうすればいいですか。'}, {'gender': 'female', 'context': 'はい、会議室の利用には事前に申し込みが必要です。利用希望日はいつですか。'}, {'gender': 'male', 'context': '来週の火曜日です。'}, {'gender': 'female', 'context': '来週の火曜日ですね。利用時間は何時から何時までですか。'}, {'gender': 'male', 'context': '午後１時から３時までお願いします。'}, {'gender': 'female', 'context': 'かしこまりました。では、申込書に必要事項を書いて、受付に出してください。'}, {'gender': 'male', 'context': 'はい、わかりました。'}, {'gender': 'female', 'context': 'あと、利用後は机や椅子を元の位置に戻して、部屋をきれいにしてくださいね。'}, {'gender': 'male', 'context': 'はい、気をつけます。ありがとうございます。'}]}

Processing 第３部　聴解: 

Total execution time: 204.78 seconds


In [13]:
# yaml_str = yaml.dump(output_data, allow_unicode=True, sort_keys=False, default_flow_style=False)
# # Display as formatted YAML
# display(Markdown(yaml_str))

In [14]:
output_data

{'sections': [{'section_title': '第１部\u3000語彙・文法',
   'subsections': [{'subsection_title': '(sentence_structure)',
     'description': '問題8：1問。つぎの文章を読んで、文章全体の内容を考えて、文中の 19 から 22 の中に入る最もよいものを、１・２・３・４から一つえらびなさい。',
     'question_topics': [{'topic': 'しゅうしょく',
       'result': {'html_article': "<div class='article'><h3>初めての就職活動</h3><p>私は大学三年生の時、初めて<strong>就職活動</strong>を始めました。就職活動はとても大変だと聞いていましたが、<strong>【19】</strong>、自分の将来について考える良いきっかけにもなりました。</p><p>最初はどの会社に応募したらいいか分からず、インターネットでいろいろな企業について調べました。説明会に参加したり、先輩にアドバイスをもらったりして、少しずつ自分に合う会社が分かってきました。<strong>【20】</strong>、面接はとても緊張しましたが、何度も練習することで自信がつきました。</p><p>私は「人の役に立つ仕事がしたい」と思っていたので、社会福祉の会社に応募しました。面接では、自分の考えや経験を<strong>【21】</strong>話すように心がけました。</p><p>最終的に、希望していた会社から内定をもらうことができました。今は社会人として働きながら、毎日新しいことを学んでいます。就職活動は大変でしたが、<strong>【22】</strong>、とても良い経験になったと思います。</p></div>",
        'questions': [{'html_question': '19.',
          'correct_answer': 2,
          'choices': ['もちろん', 'けれども', 'たとえば', 'それとも']},
         {'html_question': '20.',
          

In [15]:
def render_to_html(sections):
    html = '<html><body style="text-align: left;">\n'
    for section in sections:
        html += f'<h1>{section.get("section_title", "")}</h1>\n'
        for subsection in section.get("subsections", []):
            html += f'<h2>{subsection.get("subsection_title", "")}</h2>\n'
            if "description" in subsection:
                html += f'<p>{subsection["description"]}</p>\n'
            for idx, qtopic in enumerate(subsection.get("question_topics", []), 1):
                result = qtopic.get("result", {})
                if isinstance(result, dict):
                    if "html_article" in result:
                        html += result["html_article"] + "\n"
                    if "html_question" in result:
                        html += '<div>\n'
                        # html += f'<div style="display: flex; align-items: center;"><div>{idx}.  </div>{result["html_question"]}</div>\n'
                        html += f'<p><strong>{idx}. </strong>{result["html_question"]}</p>'
                        if "choices" in result:
                            correct = result.get("correct_answer", -1)
                            html += '<ul>\n'
                            for idx, choice in enumerate(result["choices"], 1):
                                if idx == correct:
                                    html += f"<li>{idx}. <b>{choice}</b> <span style='color:green;'>(correct)</span></li>\n"
                                else:
                                    html += f"<li>{idx}. {choice}</li>\n"
                            html += '</ul>\n'
                        html += '</div>\n'
                    if "questions" in result and isinstance(result["questions"], list):
                        for q in result["questions"]:
                            html += f'<p><strong>Q: {q["html_question"]}</strong></p>\n'
                            if "choices" in q:
                                correct = q.get("correct_answer", -1)
                                html += '<ul>\n'
                                for idx, choice in enumerate(q["choices"], 1):
                                    if idx == correct:
                                        html += f"<li><b>{choice}</b> <span style='color:green;'>(correct)</span></li>\n"
                                    else:
                                        html += f"<li>{choice}</li>\n"
                                html += '</ul>\n'
                            if "explanation" in q:
                                html += f'<p><em>Explanation: {q["explanation"]}</em></p>\n'
                    if "questions" in result and isinstance(result["questions"], dict):
                        q = result["questions"]
                        html += f'<p><strong>Q: {q["html_question"]}</strong></p>\n'
                        if "choices" in q:
                            correct = q.get("correct_answer", -1)
                            html += '<ul>\n'
                            for idx, choice in enumerate(q["choices"], 1):
                                if idx == correct:
                                    html += f"<li><b>{choice}</b> <span style='color:green;'>(correct)</span></li>\n"
                                else:
                                    html += f"<li>{choice}</li>\n"
                            html += '</ul>\n'
                        if "explanation" in q:
                            html += f'<p><em>Explanation: {q["explanation"]}</em></p>\n'
    html += '</body></html>'
    return html

In [20]:
html_output = render_to_html(output_data['sections'])
display(HTML(html_output))

時期,主な準備,注意点
1か月前,新居の契約・引っ越し業者の予約,人気のある時期は早めに予約が必要
2週間前,住所変更の手続き・荷物の整理,役所や銀行で手続きをする
1週間前,荷造り・不要品の処分,ごみの日を確認して捨てる
前日,貴重品の準備・冷蔵庫の中身を整理,冷蔵庫の電源を切る
当日,新居へ移動・ガスや電気の確認,忘れ物がないか確認する


In [17]:
timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
output_folder = "output"
html_filename = f"JLPT_{timestamp}.html"
html_filepath = os.path.join(output_folder,html_filename)
with open(html_filepath, "w", encoding="utf-8") as file:
    file.write(html_output)

In [18]:
# Create a Document object
document = Document()
# Add a section to the document
section = document.AddSection()
# Set the page margins to 72 points (72 points = 1 inch)
section.PageSetup.Margins.All = 72

# Add a paragraph to the section
paragraph = section.AddParagraph()

# Add the HTML string to the paragraph
paragraph.AppendHTML(html_output)

doc_filename = f"JLPT_{timestamp}.docx"
doc_filepath = os.path.join(output_folder,doc_filename)
# Save the result document to a DOCX file
document.SaveToFile(doc_filepath, FileFormat.Docx2016)
# Or save the result document to a DOC file
# document.SaveToFile("HtmlStringToDoc.doc", FileFormat.Doc)

document.Close()

In [19]:
import paramiko

# Load the private key from the environment variable
# Parse the private key
key_file_path = 'temp_id_ed25519'
pkey = paramiko.Ed25519Key.from_private_key_file(key_file_path)

# SSH and SFTP parameters
hostname = "58.246.203.10"
username = "hwu"
port = 22

remote_html_file_path = f"/var/www/html/{html_filename}"
remote_doc_file_path = f"/var/www/html/{doc_filename}"

# Establish SSH connection
ssh = paramiko.SSHClient()
ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
ssh.connect(hostname=hostname, port=port, username=username, pkey=pkey)

# Transfer the file
sftp = ssh.open_sftp()
sftp.put(html_filepath, remote_html_file_path)
print(f"File transferred to {hostname}:{remote_html_file_path}")
sftp.put(doc_filepath, remote_doc_file_path)
print(f"File transferred to {hostname}:{remote_doc_file_path}")

# Clean up
sftp.close()
ssh.close()

2025-06-09 22:38:35,875 - INFO - Connected (version 2.0, client OpenSSH_9.6p1)
2025-06-09 22:38:37,042 - INFO - Authentication (publickey) successful!
2025-06-09 22:38:50,081 - INFO - [chan 0] Opened sftp connection (server version 3)


File transferred to 58.246.203.10:/var/www/html/JLPT_20250609_223833.html


2025-06-09 22:38:52,312 - INFO - [chan 0] sftp session closed.


File transferred to 58.246.203.10:/var/www/html/JLPT_20250609_223833.docx
